# Building a Spam Filter with Naive Bayes

## Introduction

In this project, we're going to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans.


The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](`https://archive.ics.uci.edu/ml/datasets/sms+spam+collection`). You can also download the dataset directly [from this link](`https://dq-content.s3.amazonaws.com/433/SMSSpamCollection`). 


Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

## Exploring the Dataset

In [1]:
# Reading in the dataset
# The dataset doesn't have a header row, so we set header=None
# The data points are tab separated, so we set sep='\t'
# The names parameter is used to name the columns

# Importing pandas
import pandas as pd



sms_collection = pd.read_csv('SMSSpamCollection', sep='\t', 
                             header=None, names=['Label', 'SMS'])


In [2]:
print(sms_collection.shape)
print(sms_collection['Label'].value_counts(normalize=True)*100)
sms_collection.head()

(5572, 2)
ham     86.593683
spam    13.406317
Name: Label, dtype: float64


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Training and Test set

From the output above, we saw that about 87% of the messages are ham ("ham" means non-spam), and the remaining 13% are spam. Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

* A training set, which we'll use to "train" the computer how to classify messages.
* A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

* The training set will have 4,458 messages (about 80% of the dataset).
* The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

Like stated in the introduction, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.

In [3]:
# Randomize the dataset

sms_randomized = sms_collection.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(sms_randomized) * 0.8)

# Training/Test split
training_set = sms_randomized[:training_test_index].reset_index(drop=True)
test_set = sms_randomized[training_test_index:].reset_index(drop=True)


print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


`We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.`

In [4]:
# Showing the percentage of spam and ham in both training and test sets

df = pd.DataFrame()
df['Training Set'] = training_set['Label'].value_counts(normalize = True)
df['Test Set'] = test_set['Label'].value_counts(normalize = True)
df

,Training Set,Test Set
ham,0.86541,0.868043
spam,0.13459,0.131957


The results look good!

The next big step is to use the training set to teach the algorithm to classify new messages.

Our Naive Bayes algorithm will make the classification based on the results it gets from these two equations:

![](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)
![](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)
![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Let's also summarize what the terms in the equations above mean:

\begin{aligned} &N_{w_i|Spam} = \text{the number of times the word } w_i\text{ occurs in spam messages} \\ &N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\ \\ &N_{Spam} =\text{total number of words in spam messages} \\ &N_{Spam^C} = \text{total number of words in non-spam messages} \\ \\ &N_{Vocabulary} = \text{total number of words in the vocabulary} \\ &\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter}) \end{aligned}

But, first we'll move on to cleaning the training_set dataset.

## Data Cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the training_set data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:

![](https://camo.githubusercontent.com/27a4a0a699bd8f0713d73347abe2929c267a03d5/68747470733a2f2f64712d636f6e74656e742e73332e616d617a6f6e6177732e636f6d2f3433332f637067705f646174617365745f332e706e67)

## Letter Case and Punctuation

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [5]:
# Before cleaning
training_set.head(3)

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired


In [6]:
# Removing the punctuation from the SMS column of the training_set dataframe
# Bringing every letter to lowercase

training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()


# After cleaning
training_set.head(3)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired


## Creating the Vocabulary

The end goal with this data cleaning process is to bring our training set to the format shown above. In the format, we can see that with the exception of the `"Label"` column, every other column in the transformed table above represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message). We call the set of unique words a **vocabulary**


Now, we'll create a vocabulary for the messages in the training set. The vocabulary should be a Python list containing all the unique words across all messages, where each word is represented as a string.


In [7]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []

for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)

        
# transforming the vocabulary list into a set to remove duplicates
vocabulary_set =  set(vocabulary)

# transforming the vocabulary set back into a list
vocabulary = list(vocabulary_set)
print(len(vocabulary))

7783


`It looks like there are 7,783 unique words in all the messages of our training set.`

## The Final Training Set

Now we're going to use the vocabulary to make the data transformation we need. 

We'll first build a dictionary named `word_counts_per_sms`, where 
* each `key` is a unique word (a string) from the vocabulary, and 
* each `value` is a list of the length of ``training_set['SMS']``, where each element in the list is a 0

We loop over ``training_set['SMS']`` using at the same time the `enumerate() function` to get both the index and the SMS message (index and sms)
* Using a nested loop, we loop over sms (where sms is a list of strings, where each string represents a word in a message).
    * We incremenent ``word_counts_per_sms[word][index]`` by 1



In [8]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) 
                       for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

`Now that we have the dictionary we need, lets convert it to a dataframe.`

In [9]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


`Finally, we'll concatenate the DataFrame we just built above with the DataFrame containing the training set (this way, we'll also have the Label and the SMS columns)`

In [10]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

![](https://render.githubusercontent.com/render/math?math=P%28Spam%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Spam%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CSpam%29&mode=display)
![](https://render.githubusercontent.com/render/math?math=P%28Ham%20%7C%20w_1%2Cw_2%2C%20...%2C%20w_n%29%20%5Cpropto%20P%28Ham%29%20%5Ccdot%20%5Cprod_%7Bi%3D1%7D%5E%7Bn%7DP%28w_i%7CHam%29&mode=display)

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)
![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

* P(Spam) and P(Ham)
* NSpam, NHam, NVocabulary

We'll also use Laplace smoothing and set $\alpha = 1$.

In [11]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

The parameters :

* P(wi|spam)
* P(wi|ham)
where wi stands for each word in the vocabulary. 

Also, to calculate P(wi|Spam) and P(wi|Ham), we'll use these equations:

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)
![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CHam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CHam%7D%20%2B%20%5Calpha%7D%7BN_%7BHam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)


P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message. For instance: the steps we take to calculate P("secret"|Spam) will be identical for any new message that contains the word "secret" in the given the message label (which is `Spam` in this instance). 

This means that we can use our training set to calculate the probability for each word in our vocabulary. We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

Two separate dictionaries, for spam and ham (non-spam) are maintained, mapping each word to its probability of appearing in the respective label (Spam and Ham).

In [12]:
# Initializing the parameters for both Spam and Ham with two dictionaries, 
# one dictionary to store the parameters for P(wi|Spam) and the other for P(wi|Ham), 
# where each key-value pair is a unique word (from our vocabulary) represented as a string and the value is 0.

parameters_spam = {unique_word: [0] for unique_word in vocabulary}
parameters_ham = {unique_word: [0] for unique_word in vocabulary}

# spam_messages and ham_messages have already been isolated in the cell above

# Calculate the probabilities
for word in vocabulary:
    sum_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (sum_word_given_spam + alpha)/(n_spam + n_vocabulary)
    parameters_spam[word] += p_word_given_spam
    
    sum_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (sum_word_given_ham + alpha)/(n_ham + n_vocabulary)
    parameters_ham[word] += p_word_given_ham
    


## Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new `message (w1, w2, ..., wn)`:
     >* The input variable `message` is assumed to be a string
     * A bit of data cleaning is done on the string `message`:
        * We remove the `punctuation` using the `re.sub() function`
        * We bring all letters to `lower case` using the `str.lower() method`
        * We `split` the string at the `space character` and transform it into a Python `list` using the `str.split() method`


* Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn) by:
     >* We create two variables: `p_spam_given_message` *(i.e P(Spam|w1, w2, ..., wn)* and `p_ham_given_message` *(i.e P(Ham|w1, w2, ..., wn)* and assign them with an initial value. 
       * We initiate the variables as `p_spam_given_message = p_spam and p_ham_given_message = p_ham` *(p_spam and p_ham are P(Spam) and P(Ham) respectively, and they were calculated earlier).*
     
     >* Iterating over each word in `message` (the input of the spam filter function), which should be a list of strings by the time we start this loop. For each word:
       * If the word is present in the dictionary containing the `spam parameters (parameters_spam)`, then *update the value of p_spam_given_message by multiplying with the parameter value specific to that word.* 
       * If the word is present in the dictionary containing the `ham parameters (parameters_ham)`, then *update the value of p_ham_given_message by multiplying with the parameter value specific to that word.*
       * If the word is not present in any of the two dictionaries, then *we don't do anything.* Note that some new messages will contain words that are not part of the vocabulary. Recall that we simply ignore words that are not part of the vocabulary.
         

* Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
     >* If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
     * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
     * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [13]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()


    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

`Let's now test out our classify() function with two new messages:`

In [14]:
message_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'

classify(message_1)

P(Spam|message): [1.34812902e-25]
P(Ham|message): [1.9368049e-27]
Label: Spam


In [15]:
message_2 = "Sounds good, Tom, then see u there"

classify(message_2)

P(Spam|message): [2.43723757e-25]
P(Ham|message): [3.68753044e-21]
Label: Ham


## Measuring the Spam Filter's Accuracy

The results of using the `classify()` function above on two sample messages, looks promising. Now, lets try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

First off, we'll change the `classify()` function that we wrote previously to return the labels instead of printing them.

Then, we can use the newly modified function to create a new column in our test set.

Finally, we do a comparison between the predicted values and the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}


In [16]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

`Now we use the classify_test_set() function to create a new column in our test set`

In [17]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


`Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages`

In [18]:
# Measuring the accuracy of the spam filter
# Initializing the variable total with the number of messages in the test set

total = len(test_set)
correct = 0

for i, row in test_set.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1

        
        
print('Correct:', correct)
print('Total:', total)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)  

Correct: 1100
Total: 1114
Incorrect: 14
Accuracy: 0.9874326750448833


`The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.`

# Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set we used, which is a pretty good result. Our initial goal was an accuracy of over 80%, and we managed to do way better than that.